# Caso Práctico - ENGIE

*Jose Enrique Zafra Mena*

## Introducción


## Carga de datos

In [80]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl

In [126]:
facturas_df=pd.read_excel('facturas.xlsx')
mapping_df=pd.read_excel('Mapping.xlsx')

# renombramos los Deals
CTV_df=pd.read_excel('JVLNG_CTV_STOK_MARZO_2024_05_23_14_12.xlsx')
IAC_df=pd.read_excel('MEDG2_IAC_GETRA_MARZO_2024_05_23_14_12.xlsx')
AOC2_df=pd.read_excel('SUGS2_AOC_STOK_MARZO_2024_05_23_14_12.xlsx')
AOC_df=pd.read_excel('SUGST_AOC_STOK_MARZO_2024_05_23_14_12.xlsx')


In [63]:
mapping_df.head(100)

,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA
1,TVB,PLANTAS,JVLNG,ICC,GETRA
2,TVB,PLANTAS,JVLNG,ICC,GETRA
3,TVB,PLANTAS,JVLNG,ICC,GETRA
4,TVB,PLANTAS,JVLNG,ICC,GETRA
5,TVB,PLANTAS,JVLNG,ICC,GETRA
6,TVB,PLANTAS,JVLNG,ICC,GETRA
7,TVB,PLANTAS,JVLNG,ICC,GETRA
8,TVB,PLANTAS,JVLNG,ICC,GETRA
9,TVB,PLANTAS,JVLNG,ICC,GETRA


## Transformación de los datos


### Mapping
Quitamos todas las Commoditys que no necesitamos: nos quedamos solo con CTV, IAC, AOC.

In [130]:
# QUITAMOS LOS DUPLICADOS

mapping_df=mapping_df.query("Commodity in ['CTV', 'IAC', 'AOC'] and Portfolio in ['MEDG2','JVLNG', 'SUGST', 'SUGS2']").drop_duplicates()

mapping_df.head(100)

,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA
16,TVB,Almacenamiento TVB,JVLNG,CTV,STOK
25,AVB,Servicio agregado AVB,SUGST,AOC,STOK
26,AVB,ExtracciÃ³n,SUGST,AOC,STOK
27,AVB,Salida PVB a AVB,SUGST,AOC,STOK
28,AVB,InyecciÃ³n,SUGST,AOC,STOK
29,AVB,Entrada PVB desde AVB,SUGST,AOC,STOK
30,AVB,Almacenamiento AVB,SUGST,AOC,STOK
31,AVB,Servicio agregado AVB,SUGS2,AOC,STOK
32,AVB,ExtracciÃ³n,SUGS2,AOC,STOK


### Facturas
Quitamos columnas que no son relevantes, ajustamos formatos de las fechas, y reordenamos columnas:

In [82]:
mapping_df.rename(columns={'Servicio facturado': 'Servicio Facturado'}, inplace=True)

facturas_new_df = facturas_df.loc[:, ['NumeroFactura', 'Importe', 'FechaFactura', 'Origen', 'ServicioFacturado']]

facturas_new_df.rename(columns={'NumeroFactura': 'Número de Factura', 'Importe':'Importe Factura', 'FechaFactura':'Fecha Factura',
                                'ServicioFacturado':'Servicio Facturado'}, inplace=True)

facturas_new_df['Fecha Factura'] = pd.to_datetime(facturas_new_df['Fecha Factura'], dayfirst=True, format="%d/%m/%y")

#facturas_new_df['Fecha Factura 2'] = facturas_new_df['Fecha Factura'].dt.strftime('%d/%m/%Y')

facturas_new_df.head()

,Número de Factura,Importe Factura,Fecha Factura,Origen,Servicio Facturado
0,2024264672,123195.13,2024-03-04,AVB,Servicio agregado AVB
1,2024264671,111963.62,2024-03-04,AVB,Servicio agregado AVB
2,2324004231,503763.47,2024-03-06,C.I. Almería,Entrada PVB
3,2024164592,1375.12,2024-03-04,TVB,Almacenamiento TVB
4,2024164591,1787.43,2024-03-04,TVB,Almacenamiento TVB


### Deals
Los Deals tienen columnas que no necesitamos para la conciliación. Necesitamos solo algo que lo identifique (ID), su importe y una categoría

In [83]:
# para renombrar las columnas, usamos las columnas de CTV
CTV_min_df = CTV_df.loc[:,['Id', 'TotalQuantity', 'TradeDate', 'CommodityType']]

# renombramos
CTV_min_df.rename(columns={'TotalQuantity':'Quantity'} , inplace=True)

# ahora escogemos las columnas que queremos de cada Deal
IAC_min_df = IAC_df.loc[:,['Id', 'Quantity', 'TradeDate', 'CommodityType']]
AOC_min_df = AOC_df.loc[:,['Id', 'Quantity', 'TradeDate', 'CommodityType']]
AOC2_min_df = AOC2_df.loc[:,['Id', 'Quantity', 'TradeDate', 'CommodityType']]


Vamos a conectar el Mapping con cada Deal, ya que queremos comparar con las facturas.

Para conectar Mapping con CTV usamos Commodity==CTV

Para conectar Mapping con IAC usamos Commodity==IAC

Para conectar Mapping con AOC usamos Commodity==AOC & Portofolio: SUGST

Para conectar Mapping con AOC2 usamos Commodity==AOC & Portofolio: SUGS2


In [84]:
# para poder unir cada Deal con Mapping individualmente, añadimos las columnas Portofolio y Commodity a cada deal
# y le asignamos su nombre correspondiente

CTV_min_df['Portfolio']='JVLNG'
CTV_min_df['Commodity']='CTV'


IAC_min_df['Portfolio']='MEDG2'
IAC_min_df['Commodity']='IAC'

AOC_min_df['Portfolio']='SUGST'
AOC_min_df['Commodity']='AOC'

AOC2_min_df['Portfolio']='SUGS2'
AOC2_min_df['Commodity']='AOC'

AOC_min_df.head()

,Id,Quantity,TradeDate,CommodityType,Portfolio,Commodity
0,41888250,800000.000,2024-02-22T00:00:00+01:00,EG,SUGST,AOC
1,38290460,924.588,2023-04-20T00:00:00+02:00,EG,SUGST,AOC
2,38249489,352.340,2023-04-17T00:00:00+02:00,EG,SUGST,AOC
3,38054569,3910.694,2023-03-24T00:00:00+01:00,EG,SUGST,AOC
4,36525979,800000.000,2022-11-16T00:00:00+01:00,EG,SUGST,AOC


In [86]:
# ahora los unimos todos y camb
deals_df=pd.concat([CTV_min_df, IAC_min_df, AOC_min_df, AOC2_min_df], ignore_index=True)


# vemos si hay algún ID duplicado

# cambiamos el formato de fecha
#todo

deals_df.head(100)

,Id,Quantity,TradeDate,CommodityType,Portfolio,Commodity
0,42563586,90000.0,2024-04-29T00:00:00+02:00,EG,JVLNG,CTV
1,42518127,50000.0,2024-04-25T00:00:00+02:00,EG,JVLNG,CTV
2,42518126,50000.0,2024-04-25T00:00:00+02:00,EG,JVLNG,CTV
3,42518125,50000.0,2024-04-25T00:00:00+02:00,EG,JVLNG,CTV
4,42518124,50000.0,2024-04-25T00:00:00+02:00,EG,JVLNG,CTV
...,...,...,...,...,...,...
58,37671348,900.0,2023-02-22T00:00:00+01:00,EG,SUGS2,AOC
59,37356869,400000.0,2022-11-21T00:00:00+01:00,EG,SUGS2,AOC
60,37186479,100000.0,2023-01-12T00:00:00+01:00,EG,SUGS2,AOC
61,36876402,100000.0,2022-12-19T00:00:00+01:00,EG,SUGS2,AOC


#### Merge con mapping

In [139]:
# ahora unimos cada uno con mapping
deals_mapped_df = (pd.merge(deals_df, mapping_df,on=['Portfolio', 'Commodity'], how='inner').drop_duplicates())

test=deals_mapped_df.shape[0]-deals_df.shape[0]

print('test',test)

#deals_mapped_df.head(1000)
deals_mapped_df.head(1000)

## SALEN DE MÁS PORQUE LA RELACIÓN NO ES UNICA, PARA UN MISMO CANAL (PORTOFOLIO+COMMODITY), 
## HAY DISTINTOS SERVICIOS FACTURADOS

test 95


,Id,Quantity,TradeDate,CommodityType,Portfolio,Commodity,Origen,Servicio facturado,DealType
0,42563586,90000.0,2024-04-29T00:00:00+02:00,EG,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
1,42518127,50000.0,2024-04-25T00:00:00+02:00,EG,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
2,42518126,50000.0,2024-04-25T00:00:00+02:00,EG,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
3,42518125,50000.0,2024-04-25T00:00:00+02:00,EG,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
4,42518124,50000.0,2024-04-25T00:00:00+02:00,EG,JVLNG,CTV,TVB,Almacenamiento TVB,STOK
...,...,...,...,...,...,...,...,...,...
153,36572988,100000.0,2022-11-21T00:00:00+01:00,EG,SUGS2,AOC,AVB,ExtracciÃ³n,STOK
154,36572988,100000.0,2022-11-21T00:00:00+01:00,EG,SUGS2,AOC,AVB,Salida PVB a AVB,STOK
155,36572988,100000.0,2022-11-21T00:00:00+01:00,EG,SUGS2,AOC,AVB,InyecciÃ³n,STOK
156,36572988,100000.0,2022-11-21T00:00:00+01:00,EG,SUGS2,AOC,AVB,Entrada PVB desde AVB,STOK


## Análisis de los datos

### Facturas

In [15]:
# Número total de facturas
numero_total_facturas = facturas_new_df['Número de Factura'].count()
print('Número total de facturas: ',numero_total_facturas)
# Importe total de facturas 
importe_total_facturas = facturas_new_df['Importe Factura'].sum()
print('Importe total de facturas: ',importe_total_facturas)

# Distribución por categoría

### NO APLICABLE CREO

#facturas_new_df.head(15)

Número total de facturas:  14
Importe total de facturas:  811357.9799999999


,Número de Factura,Importe Factura,Fecha Factura,Origen,Servicio Facturado
0,2024264672,123195.13,2024-03-04,AVB,Servicio agregado AVB
1,2024264671,111963.62,2024-03-04,AVB,Servicio agregado AVB
2,2324004231,503763.47,2024-03-06,C.I. Almería,Entrada PVB
3,2024164592,1375.12,2024-03-04,TVB,Almacenamiento TVB
4,2024164591,1787.43,2024-03-04,TVB,Almacenamiento TVB
5,2024164590,1930.23,2024-03-04,TVB,Almacenamiento TVB
6,2024164589,1472.13,2024-03-04,TVB,Almacenamiento TVB
7,2024164588,1965.66,2024-03-04,TVB,Almacenamiento TVB
8,2024164587,1041.67,2024-03-04,TVB,Almacenamiento TVB
9,2024164586,1030.97,2024-03-04,TVB,Almacenamiento TVB


### Deals

In [ ]:
# Número total de Deals

# Volumen total de Deals

# Distribución por Categorías

## Conciliación

### Merge de Facturas con los *Deals*

### Unificación de fechas y filtrado

In [89]:
# por ultimo cambio el nombre y el orden de las columnas para verlo mejor
facturas_conciliadas_Marzo_df.rename(columns={'NumeroFactura': 'Número de Factura', 'FechaFactura':'Fecha Factura', 'Id':'ID Deal', 'LegId':'LegID', 
                                              'Quantity':'Importe Deal', 'Importe':'Importe Factura', 'TradeDate':'Fecha Deal'}, inplace=True)

orden = ['Número de Factura', 'ID Deal', 'Importe Factura', 'Importe Deal', 'Fecha Factura', 'Fecha Deal', 'LegID',
         'Origen', 'Servicio Facturado', 'LegType', 'Category', 'Portfolio', 'Commodity', 'DealType', 'Conciliado', 'Observaciones']

facturas_conciliadas_Marzo_df = facturas_conciliadas_Marzo_df.reindex(columns=orden)

facturas_conciliadas_Marzo_df.head(100)

,Número de Factura,ID Deal,Importe Factura,Importe Deal,Fecha Factura,Fecha Deal,LegID,Origen,Servicio Facturado,LegType,Category,Portfolio,Commodity,DealType,Conciliado,Observaciones
192,2024164592,42238975,1375,15000,2024-04-03,2024-03-25 00:00:00+00:00,42238975,TVB,Almacenamiento TVB,Delivery,O,JVLNG,CTV,STOK,False,
195,2024164592,42238679,1375,45000,2024-04-03,2024-03-25 00:00:00+00:00,42238679,TVB,Almacenamiento TVB,Delivery,O,JVLNG,CTV,STOK,False,
198,2024164592,42238678,1375,30000,2024-04-03,2024-03-25 00:00:00+00:00,42238678,TVB,Almacenamiento TVB,Delivery,O,JVLNG,CTV,STOK,False,
201,2024164592,42238677,1375,31000,2024-04-03,2024-03-25 00:00:00+00:00,42238677,TVB,Almacenamiento TVB,Delivery,O,JVLNG,CTV,STOK,False,
204,2024164592,42238676,1375,91000,2024-04-03,2024-03-25 00:00:00+00:00,42238676,TVB,Almacenamiento TVB,Delivery,O,JVLNG,CTV,STOK,False,
318,2024164591,42238975,1787,15000,2024-04-03,2024-03-25 00:00:00+00:00,42238975,TVB,Almacenamiento TVB,Delivery,O,JVLNG,CTV,STOK,False,
321,2024164591,42238679,1787,45000,2024-04-03,2024-03-25 00:00:00+00:00,42238679,TVB,Almacenamiento TVB,Delivery,O,JVLNG,CTV,STOK,False,
324,2024164591,42238678,1787,30000,2024-04-03,2024-03-25 00:00:00+00:00,42238678,TVB,Almacenamiento TVB,Delivery,O,JVLNG,CTV,STOK,False,
327,2024164591,42238677,1787,31000,2024-04-03,2024-03-25 00:00:00+00:00,42238677,TVB,Almacenamiento TVB,Delivery,O,JVLNG,CTV,STOK,False,
330,2024164591,42238676,1787,91000,2024-04-03,2024-03-25 00:00:00+00:00,42238676,TVB,Almacenamiento TVB,Delivery,O,JVLNG,CTV,STOK,False,


## Propuesta de Valor